This code is to classify the selected examples automatically for those criteria that allow automatic classification.

In [1]:
import pandas as pd
import xlrd
import re
from re import search

In [40]:
# opens the excel sheet and reads it
# excel sheet needs to be in the same working directory as this file

file = 'data_1900_raw.xls'
data = pd.ExcelFile(file)

 # returns the all the sheet names within the excel file
print(data.sheet_names)

['Sheet1']


In [41]:
# parses the sheet into a data frame to show the column structure within the file

data_frame = data.parse('Sheet1')
data_frame.info

# only shows the first 10 rows
data_frame.head(10)

,Unnamed: 0,ID,Author,Title,Date,Genre,occurrence,wh,int,Interrogatives
0,0,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Blanche ?
1,1,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Maman ?
2,2,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Est-ce fini ?
3,3,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,As-tu fait le compte de ton côté ?
4,4,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Combien de personnes serons-nous à table ?
5,5,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Crois-tu que le dîner sera meilleur parce que ...
6,6,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Mais es-tu bien sûre que ce soit l'usage ?
7,7,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Veux-tu que nous arrêtions les places ensemble ?
8,8,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Madame de Saint-Genis ?
9,9,text_114,"BECQUE, Henry",LES CORBEAUX,1924,Drame,NaN,NaN,NaN,Son fils ?


In [42]:
# reads in the spreadsheet data

workbook = xlrd.open_workbook('data_1900_raw.xls')
sheet = workbook.sheet_by_name('Sheet1')

# gets the first sheet
sheet_1 = workbook.sheet_by_index(0)

row_count = sheet.nrows
col_count = sheet.ncols
print(f'Total rows: {row_count}\nTotal columns: {col_count}')

Total rows: 1777
Total columns: 10


We're now going to create a new list of lists out of the file, and populate the empty columns with as much data as possible.

The data that should be easy to classify automatically are: est-ce que / wh- at the beginning or end of sentence / presence of c'est.

In [43]:
# initialises list of lists
all_rows = []

for rx in range(sheet.nrows): # rx is an int, row is a list 
    row = sheet.row(rx)

    # initialises list that will contain our data
    data_list = []

    # populates list that stores all rows
    data_list.append(row[0].value)
    data_list.append(row[1].value)
    data_list.append(row[2].value) 
    data_list.append(row[3].value) 
    data_list.append(row[4].value) 
    data_list.append(row[5].value) 
    data_list.append(row[6].value) # occurrence
    data_list.append(row[7].value) # wh
    data_list.append(row[8].value) # int
    data_list.append(row[9].value) # segment

    all_rows.append(data_list)
    # print(all_rows)
    
# prints extracted rows
print(f'We created a list that embeds {len(all_rows)} lists.')

We created a list that embeds 1777 lists.


In [58]:
# iterates through list of lists and populates empty cells

wh1 = 'comment'
wh2 = 'quand'
wh3 = 'qui'
wh4 = 'où'
wh5 = 'quoi'
wh6 = 'qu\''
wh7 = 'pourquoi'
wh8 = 'que\s'

unwanted1 = 'qu\'on'
unwanted2 = 'qu\'elle'
unwanted3 = 'qu\'il'
unwanted4 = 'qu\'une'
unwanted5 = 'qu\'un'
unwanted6 = 'qu\'à'
unwanted7 = 'qu\'au'
unwanted8 = 'qu\'aux'
unwanted9 = 'qu\'avant'

string = "est-ce"
string_count = 0
string2 = 'c\'est'
string2_count = 0

ex_situ_count = 0
in_situ_count = 0
fragment_count = 0
sv_count = 0

for list in all_rows:
    text = list[9] # index 9 is where text of interest is stored, returns a cell
    text = text.lower()

    if search(string, text):
        list[7] = 'ex situ'
        list[8] = 'est-ce que'
        string_count += 1
    elif re.search('^comment\sje', text, flags=0) or re.search('^comment\stu', text, flags=0) or re.search('^comment\selle', text, flags=0) or re.search('^comment\sil', text, flags=0) or re.search('^comment\snous', text, flags=0) or re.search('^comment\svous', text, flags=0) or re.search('^comment\selles', text, flags=0) or re.search('^comment\sils', text, flags=0) or re.search('^comment\sça', text, flags=0):
        list[8] = 'SV'
        list[7] = 'ex situ'
    elif re.search('^quand\sje', text, flags=0) or re.search('^quand\stu', text, flags=0) or re.search('^quand\selle', text, flags=0) or re.search('^quand\sil', text, flags=0) or re.search('^quand\snous', text, flags=0) or re.search('^quand\svous', text, flags=0) or re.search('^quand\selles', text, flags=0) or re.search('^quand\sils', text, flags=0) or re.search('^quand\sça', text, flags=0):
        list[8] = 'SV'
        list[7] = 'ex situ'
    elif re.search('^qui\sje', text, flags=0) or re.search('^qui\stu', text, flags=0) or re.search('^qui\selle', text, flags=0) or re.search('^qui\sil', text, flags=0) or re.search('^qui\snous', text, flags=0) or re.search('^qui\svous', text, flags=0) or re.search('^qui\selles', text, flags=0) or re.search('^qui\sils', text, flags=0) or re.search('^qui\sça', text, flags=0):
        list[8] = 'SV'
        list[7] = 'ex situ'
    elif re.search('^où\sje', text, flags=0) or re.search('^où\stu', text, flags=0) or re.search('^où\selle', text, flags=0) or re.search('^où\sil', text, flags=0) or re.search('^où\snous', text, flags=0) or re.search('^où\svous', text, flags=0) or re.search('^où\selles', text, flags=0) or re.search('^où\sils', text, flags=0) or re.search('^où\sça', text, flags=0):
        list[8] = 'SV'
        list[7] = 'ex situ'
    elif re.search('^pourquoi\sje', text, flags=0) or re.search('^pourquoi\stu', text, flags=0) or re.search('^pourquoi\selle', text, flags=0) or re.search('^pourquoi\sil', text, flags=0) or re.search('^pourquoi\snous', text, flags=0) or re.search('^pourquoi\svous', text, flags=0) or re.search('^pourquoi\selles', text, flags=0) or re.search('^pourquoi\sils', text, flags=0) or re.search('^pourquoi\sça', text, flags=0):
        list[8] = 'SV'
        list[7] = 'ex situ'
    else:
        list[8] = 'VS'
        list[7] = 'ex situ'

    if wh1 in text:
        list[6] = wh1
    elif wh2 in text:
        list[6] = wh2
    elif wh3 in text:
        list[6] = wh3
    elif wh4 in text:
        list[6] = wh4
    elif wh5 in text:
        list[6] = wh5
    elif wh6 in text:
        list[6] = wh6
    elif wh7 in text:
        list[6] = wh7
    elif wh8 in text:
        list[6] = wh8
        list[7] = 'ex situ'
        list[8] = 'VS'
    else:
        list[6] = '---' # empty cells when there is no wh-word
        list[7] = '---'
        list[8] = '---'

    if unwanted1 in text or unwanted2 in text or unwanted3 in text or unwanted4 in text or unwanted5 in text or unwanted6 in text or unwanted7 in text or unwanted8 in text or unwanted9 in text: # eliminates cases of non interrogative "qu"
        list[6] = '---'
        list[7] = '---'
        list[8] = '---'

    if search(string2, text):
        list[8] = 'cleft'
        string2_count += 1

        if re.search('^comment\s', text, flags=0):
            list[7] = 'ex situ'
        elif re.search('^quand\s', text, flags=0):
            list[7] = 'ex situ'
        elif re.search('^qui\s', text, flags=0):
            list[7] = 'ex situ'
        elif re.search('^où\s', text, flags=0):
            list[7] = 'ex situ'
        elif re.search('^pourquoi\s', text, flags=0):
            list[7] = 'ex situ'
        elif re.search('^que\s', text, flags=0):
            list[7] = 'ex situ'
            list[8] = 'VS'
        else:
            list[7] = 'in situ'

    if re.search('^comment\s', text, flags=0):
        list[7] = 'ex situ'
    elif re.search('^quand\s', text, flags=0):
        list[7] = 'ex situ'
    elif re.search('^qui\s', text, flags=0):
        list[7] = 'ex situ'
    elif re.search('^où\s', text, flags=0):
        list[7] = 'ex situ'
    elif re.search('^pourquoi\s', text, flags=0):
        list[7] = 'ex situ'
    elif re.search('^que\s', text, flags=0):
        list[6] = 'que'
        list[7] = 'ex situ'
        list[8] = 'VS'
    ex_situ_count += 1

    if text.endswith('comment ?') or text.endswith('quand ?') or text.endswith('qui ?') or text.endswith('où ?') or text.endswith('pourquoi ?') or text.endswith('quoi ?'):
        list[7] = 'in situ'
        list[8] = 'SV'
        in_situ_count += 1

We will now create a new .xlsx file with all the classified data.

In [59]:
# creates spreadsheet using each nested list as a row

df = pd.DataFrame(data=all_rows)

# converts into excel
df.to_excel("data_1900_preclassified.xlsx", index=False)

print("Dictionary converted into excel...")

Dictionary converted into excel...
